# Exploring Stack Exchance

While everyone *loves* a fun dataset to explore, good data is expensive. It costs a significant amount of resources to generate, accurately curate, securely store, and provide robust access to. For instance, our cold-storage tape archive, [Ranch](https://www.tacc.utexas.edu/systems/ranch), grows at a rate of 8.5PB (~5.3%) per year. Despite these costs, data is often invaluable to both users and administrators.

Today, we will be exploring data from Stack Exchange. While this is probably not the kind of data you interact with on a daily basis, everyone at this camp should have some familiarity from interacting with *at least* one [Stack Exchange Community](https://stackexchange.com/sites):

- StackOverflow
- Super User
- TeX - LaTeX
- ...and more

Today, you will be using python to explore question and answer history from the Stack Exchange site of your choice. This data will be accessed over their public API. This is their **actual** data, and these methods can be extended to a variety of other datasets and websites.

## Objectives

- Use python [requests](http://docs.python-requests.org/en/master/) to download data
- Import data into [Pandas](http://pandas.pydata.org/)
- Explore data
  - Inspect and summarize data
  - Group records
  - Select and subset records
  - Visualize selection
  - Join two datasets together

## Dependencies

We will be using the following non-standard python libraries:
- [**requests** library](http://docs.python-requests.org/en/master/) *\(Already Installed\)*
- [**pandas** library](http://pandas.pydata.org/) *\(Already Installed\)*

In [ ]:
# Import necessary Libraries
import requests, json
import pandas as pd
# Render matplotlib in the notebook
%matplotlib inline

# Stack Exchange Questions

Stack Exchange has a [well documented API](https://api.stackexchange.com/), which contains enpoints for **each site**. You can perform any graphical interaction through the API while authenticated, but general information can also be retrieved anonymously. Just make sure you do not make more than 10,000 requests per day. (*I did while devloping this notebook*)

Beginning with the initial questsions submitted by users, take a look at the

[Questions API](https://api.stackexchange.com/docs/questions)

webapp on the Stack Exchange site, and build a query that you would like to use with Python.

## Goals

- Choose a site (default is StackOverflow)
- Choose Start and/or End Date
- Sort by creation
- Limit the number of questions to 10 (`pagesize`)

# Make API Request

In [ ]:
# API URL
url = 'https://api.stackexchange.com/2.2/questions'

params = dict(
    site='stackoverflow', # stackoverflow (coding) questions
    pagesize='10',        # Number of questions to return
    fromdate='1500163200',# Get epoch time from webapp
    order='desc',
    sort='creation'
)

resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

print(json.dumps(data, indent=3))

Great! If you kept `pagesize` at 10, you should have a JSON response of 10 questions. If you decided to crank up your response size, you might have to scroll a bit.

## JSON Structure

This JSON response probably looks familiar if you have ever worked with Python dictionaries in the past. At the most basic level, a JSON is a collection of key and value pairs.

```json
{
    "key1": value1,
    "key2": value2
}
```

Instead of using a numerical index, you refer to each value with the corresponding key.

- key1
- key2

This makes both the data structure and programatic access human-readable. However, the lack of indicies makes traditional access through looping somewhat difficult.

In [ ]:
# Print first question title
############################
# Pull "items" json
#  > Pull first record
#    > Pull title
print(data['items'][0]['title'])

In [ ]:
# You need to know the exact key names to traverse it
for item in data['items']:
    # Print the question title
    print("TAGS - %s"%(item['title']))
    # Print the question tags
    print("   [%s]\n"%(", ".join(item['tags'])))

## Explore

- Try pulling out the `answer_count` for each question
- Try pulling out the `view_count` for each question
- Try pulling out the submission date.
- **Extra Credit** - [Convert the epoch time to human readable](https://stackoverflow.com/a/12400584)

# Converting to Pandas

Instead of testing you on your ability to traverse a JSON tree, the goal for today is to explore data using Pandas, so lets convert the JSON to a DataFrame.

In [ ]:
questionsDF = pd.io.json.json_normalize(data['items'])
questionsDF

[`json_normalize`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html) takes a nested JSON and flattens it into a table. In our case, it flattened each return question in the `items` list. Child JSONs like owner, which described the original submitter, now have owner as a prefix in the column name.

### JSON
```
"owner": {
            "reputation": 1,
            "user_id": 6140730,
            "user_type": "registered",
            "profile_image": "https://www.gravatar.com/avatar/efa02138df0bc1f59618c365872caed6?s=128&d=identicon&r=PG&f=1",
            "display_name": "John",
            "link": "https://stackoverflow.com/users/6140730/john"
         }
```
### Table

| Column Name | Value |
|--|--|
| owner.reputation | 1 |
| owner.user_id | 6140730 |
| owner.user_type | registered |
| owner.profile_image | https://www... |
| owner.display_name | John |
| owner.link | https://stackoverflow... |

# Exploring the Data

When we transform the JSON data into a table, using `json_normalize`, the resulting table is actually a [Pandas DataFrame.](https://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe)

A DataFrame is a 2-dimensional data structure that can store data of different types

(characters, integers, floating point values, factors, and more)

in columns. It is similar to a spreadsheet or an SQL table or the data.frame in R. A DataFrame always has an index (0-based). An index refers to the row of an element in the data structure.

You can see the **bold** index column on the left of our example.

## Viewing DataFrame Attributes

Besides having text column headers, DataFrames come with some nice attributes and methods to view specific parts of the data.

## Columns

You often need to iterate over the columns of your table, and DataFrames expose those names

In [ ]:
print(questionsDF.columns)

## Shape

You can also see how many rows and columns (rows, columns) are in your DataFrame by accessing the shape attribute.

In [ ]:
print(questionsDF.shape)

## Head

If you have ever used the `head` command on a terminal to view the first N lines of a file, the head function of a DataFrame will look familiar to you. This is great for just peeking at the data and not overflowing your window.

In [ ]:
questionsDF.head()
#questionsDF.head(2)

## Tail

There is also a tail command for looking at the last N rows of a DataFrame.

In [ ]:
questionsDF.tail()
#questionsDF.tail(2)

# Grouping Records

Many of the columns in this data, like `owner.link`, may not be immediately useful to us. With a DataFrame, you can select and group specific columns for use in a downstream analysis without losing the original.

For example, we could be interested in the `view_count` of each question. An analysis of this column could show how many people also encounter a similar problem and needed to seek help on Stack Exchange.

## Column Groups

We can pull out this single column using two methods.

In [ ]:
# Dot
print(questionsDF.view_count.head())
# Bracket
print(questionsDF['view_count'].head())

We can also produce similar statistics provided by the `summary()` function in R with the `describe()` function. This can be applied directly to our column selection as so.

In [ ]:
questionsDF['view_count'].describe()

Besides only using the first 10 questions in my example data, they're all very new, so they have very few views. Lets instead work on the latest 1,000 questions and generate the same description. Stack Exchange [limits](https://api.stackexchange.com/docs/throttle) the `pagesize` of the response to 100, so we will be pulling the first 10 pages.

In [ ]:
# Latest 1000 questions

# Params pull 100 questsions per query
params = dict(
    site='stackoverflow',
    pagesize='100',
    page='1',
    order='desc',
    sort='creation'
)

nPages = 10 #How many pages you want
data = []

import sys
print("Reading Page:")
for page in map(str, range(1,nPages+1)):
    params['page']=page # Change page number
    if int(page) > 1: sys.stdout.write(", ")
    sys.stdout.write("%s"%(page))
    data += json.loads(requests.get(url=url, params=params).text)['items']

questionsDF = pd.io.json.json_normalize(data)
# Drop the "migrated_from" columns
questionsDF = questionsDF[list(filter(lambda x: 'migrated' not in x, questionsDF.columns))]

questionsDF['view_count'].describe()

Now that we have a larger pool of data, you should check out other statistics that can be generated per column. Feel free to use another numerical column as well.

## Explore

There are a bunch of [built in](https://pandas.pydata.org/pandas-docs/stable/api.html#computations-descriptive-stats) descriptive functions, but these are good to check out.

- describe()
- nuniqe()
- value_counts()

In [ ]:
# How many unique users?
questionsDF['owner.user_id'].head()

## Two-Way Groups

If you ever want to summarize by one or more variables, you can use the `groupby` method. In our case, it would be interesting to look at `view_count` statistics of answered and unanswered questions.

In [ ]:
questionsDF.groupby('is_answered')['view_count'].describe()

We can see that while there are fewer answered questions, their view count (in my test) is almost 100% higher. Neat!

## Explore

Take some time using the `groupby` method to explore other cool trends.

- Owner reputation - Is the submitter a bot?
- Score - Is the question real?

In [ ]:
#questionsDF.groupby('is_answered')['owner.reputation'].describe()

# Selecting and Subsetting Records

You can also select a subset of the data using criteria. For example, we can select all rows that have a `view_count` greater than 5.

In [ ]:
questionsDF[questionsDF.view_count > 5]

## Explore

Experiment with the

- `>`, `<`
- `==`, `!=`
- `>=`, `<=`

operators on numerical data. If you have extra time, look for questions that contain tags that you know. The tags are actually a list, so you can search for tags using the `in` operator.

In [ ]:
# Need to use the map operation on tags
questionsDF[questionsDF.tags.map(lambda x: 'python' in x)]

# Visualizing the Results

While the tables we have been generating are nice, they still contain thousands of rows. A single figure could help visualize the data as a whole. Insead of crafting specific matplotlib calls, Pandas built a universal [`plot()` function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html) into the DataFrame object to simplify figure generation.

By stating that we want to generate a histogram with `kind='hist'`, we can look at the `view_count` fequency.

In [ ]:
questionsDF['view_count'].plot(kind='hist')
# Try increasing the resolution with the "bins" parameter
# Try a square root transform of the view count

We can also plot our two-way tables.

In [ ]:
questionsDF.groupby('is_answered')['view_count'].plot(kind='hist', legend=True)

## Explore

Try generating a few figures on your own.

# Joining Tables

You can even join two datasets. Lets grab some answers so we can try joining them to their corresponding questions.

In [ ]:
url = 'https://api.stackexchange.com/2.2/answers'
params = dict(
    site='stackoverflow',
    pagesize='100',
    page='1',
    order='desc',
    sort='creation'
)

nPages = 10 #How many pages you want
data = []

import sys
print("Reading Page:")
for page in map(str, range(1,nPages+1)):
    params['page']=page # Change page number
    if int(page) > 1: sys.stdout.write(", ")
    sys.stdout.write("%s"%(page))
    data += json.loads(requests.get(url=url, params=params).text)['items']

answersDF = pd.io.json.json_normalize(data)
answersDF.head()

## Inner Join

We can return the intersection of all questions that also map to an answer by using an inner join. Assuming we had the following example data:

```
Questions
---------------------
QuestionID 0  1  2  3
ViewCount  2  4  10 7
AnswerID   NA 1  2  3

Answers
---------------------
QuestionID 5  1  2  3
Score      3  5  3  1
AnswerID   0  1  2  3
```

An inner join would yield

```
Questions X Answers
---------------------
QuestionID 1  2  3
ViewCount  4  10 7
Score      5  3  1
AnswerID   1  2  3
```

We join both `questionsDF` and `answersDF` on the `question_id` column that they both share.

In [ ]:
merged = pd.merge(left=questionsDF, right=answersDF[['answer_id','question_id']], left_on="question_id", right_on="question_id")
print(merged.shape)
merged.head()
print(questionsDF.columns)

## Left Join

Left joins return all items from the first set, and any items from the second set that overlap with the first. This is useful if we want ALL questions returned, and any questions that also match.

Using the table from the first example, a left join would yield

```
Questions LJ Answers
---------------------
QuestionID 0  1  2  3
ViewCount  2  4  10 7
Score      NA 5  3  1
AnswerID   NA 1  2  3
```

Notice that whenever there is no match on the right, fields are filled in as NA.

In [ ]:
merged = pd.merge(left=questionsDF, right=answersDF, left_on="question_id", right_on="question_id", how="left")
print(merged.shape)
merged.head()

## Explore

There are also Right and Outer joins to explore. Take a look at [the documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging) and see if you can discover anythign fun.

In [ ]:
# Try joining some data